<a href="https://colab.research.google.com/github/FabianaAlbuquerque97/AI-Fuzzy-Systems-and-Bayesian-Prediction/blob/main/Sistema_Fuzzy_Risco_Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema Fuzzy para Análise de Risco em Projetos de Software
Trabalho de Inteligência Artificial - Questão 2

## 2.1 Construção e modelagem da base de regras
**Base de dados:**
Foi gerada uma base sintética com 1000 registros simulando projetos de software, considerando as variáveis:
- Experiência da equipe (0-10)
- Tempo estimado (1-52 semanas)
- Orçamento (10-500 mil)
- Complexidade do projeto (0-10)
- Risco (0-10, calculado a partir das variáveis com base em lógica heurística)

**Geração de regras:**
Uma árvore de decisão (DecisionTreeClassifier) foi treinada para prever categorias de risco (baixo, médio, alto). A partir dos caminhos da árvore, extraímos regras fuzzy.


In [2]:
# Instalação do Scikit-Fuzzy
!pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 9.7 MB/s eta 0:00:00


In [3]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Geração da base
np.random.seed(42)
n = 1000
experience = np.random.randint(0, 11, n)
time = np.random.randint(1, 53, n)
budget = np.random.randint(10, 500, n)
complexity = np.random.randint(0, 11, n)

risk = (10 - experience) + (complexity) + (time / 13) - (budget / 100)
risk = np.clip(risk / 3.5, 0, 10)
risk_cat = np.digitize(risk, bins=[3.3, 6.6])  # 0: baixo, 1: médio, 2: alto

X = np.stack([experience, time, budget, complexity], axis=1)
y = risk_cat

# Árvore de decisão
tree = DecisionTreeClassifier(max_depth=3)
tree.fit(X, y)

DecisionTreeClassifier(max_depth=3)

## 2.2 Descrição do processo de modelagem
**Funções de pertinência:**
Usamos funções triangulares e a função `automf(3)` do Scikit-Fuzzy para gerar categorias fuzzy automaticamente: baixo, médio e alto.

**Inferência Mamdani:**
- As regras fuzzy são aplicadas sobre os valores fuzzificados.
- As saídas das regras são cortadas e agregadas com o método `max`.
- A saída é defuzzificada pelo método do centroide.


In [8]:
# Variáveis fuzzy
exp = ctrl.Antecedent(np.arange(0, 11, 1), 'experience')
tme = ctrl.Antecedent(np.arange(0, 53, 1), 'time')
bud = ctrl.Antecedent(np.arange(0, 501, 1), 'budget')
comp = ctrl.Antecedent(np.arange(0, 11, 1), 'complexity')
rsk = ctrl.Consequent(np.arange(0, 11, 1), 'risk')

exp.automf(3)
tme.automf(3)
bud.automf(3)
comp.automf(3)
rsk.automf(3)

# Regras fuzzy (baseadas na árvore de decisão)
rule1 = ctrl.Rule(exp['poor'] & comp['good'], rsk['good'])       # risco alto
rule2 = ctrl.Rule(exp['average'] & bud['average'], rsk['average'])  # risco médio
rule3 = ctrl.Rule(exp['good'] & comp['poor'], rsk['poor'])       # risco baixo

#Tempo
rule4 = ctrl.Rule(tme['poor'] & comp['good'], rsk['good'])         # tempo curto + alta complexidade → risco alto
rule5 = ctrl.Rule(tme['average'] & bud['average'], rsk['average']) # tempo médio + orçamento médio → risco médio
rule6 = ctrl.Rule(tme['good'] & bud['good'], rsk['poor'])          # tempo longo + bom orçamento → risco baixo


# Sistema de controle fuzzy
risk_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6])
risk_sim = ctrl.ControlSystemSimulation(risk_ctrl)


## 2.3 Execução da aplicação com Scikit-Fuzzy
Rodamos uma simulação com os seguintes valores de entrada:
- Experiência: 2
- Tempo: 40 semanas
- Orçamento: 100 mil
- Complexidade: 8

In [9]:
# Entradas
risk_sim.input['experience'] = 2
risk_sim.input['time'] = 40
risk_sim.input['budget'] = 100
risk_sim.input['complexity'] = 8

# Executar inferência
risk_sim.compute()
print("Risco do Projeto:", risk_sim.output['risk'])

Risco do Projeto: 5.878048780487805
